In [1]:
import requests
import pandas as pd
import datetime

# API URL
url = "https://api.openweathermap.org/data/3.0/onecall/timemachine"

# Parameters for the API request
latitude = 7.252299424314035
longitude = 80.5940338059912
api_key = "d5ec5c2bc81d15d693a895ea81cb6286"
units = "metric"

# Create an empty DataFrame to store the weather data
weather_df = pd.DataFrame(columns=["Date", "Time", "Temperature", "Feels Like", "Pressure", "Humidity", "Description"])

# Specify the start date and time
start_date = datetime.datetime(2020, 10, 18)
end_date = start_date + datetime.timedelta(hours=888)  # 888 API requests

# Iterate over the hours from the start date to the end date
for dt in pd.date_range(start_date, end_date, freq="H"):
    # Calculate the UNIX timestamp for each hour
    dt_unix = int(dt.timestamp())

    # Construct the parameters for the API request
    params = {
        "lat": latitude,
        "lon": longitude,
        "dt": dt_unix,
        "appid": api_key,
        "units": units
    }

    # Send the API request
    response = requests.get(url, params=params)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Retrieve the response data
        data = response.json()

        # Extract the required information from the response
        weather_data = data["data"][0]
        date = dt.strftime("%Y-%m-%d")
        time = dt.strftime("%H:%M:%S")
        temperature = weather_data["temp"]
        feels_like = weather_data["feels_like"]
        pressure = weather_data["pressure"]
        humidity = weather_data["humidity"]
        description = weather_data["weather"][0]["description"]

        # Add the extracted data to the DataFrame
        weather_df = weather_df.append({"Date": date, "Time": time, "Temperature": temperature, 
                                        "Feels Like": feels_like, "Pressure": pressure, 
                                        "Humidity": humidity, "Description": description}, ignore_index=True)
    else:
        print("Error: Request failed.")

# Save the DataFrame to a CSV file
weather_df.to_csv("weather_data.csv", index=False)



Weather data for 2020-10-18 00:00:00:
{'lat': 7.2523, 'lon': 80.594, 'timezone': 'Asia/Colombo', 'timezone_offset': 19800, 'data': [{'dt': 1602959400, 'sunrise': 1602980650, 'sunset': 1603023660, 'temp': 22.56, 'feels_like': 23.33, 'pressure': 1010, 'humidity': 94, 'dew_point': 21.55, 'clouds': 20, 'visibility': 10000, 'wind_speed': 0, 'wind_deg': 0, 'wind_gust': 0, 'weather': [{'id': 801, 'main': 'Clouds', 'description': 'few clouds', 'icon': '02n'}]}]}
-----------------------------------
Weather data for 2020-10-18 01:00:00:
{'lat': 7.2523, 'lon': 80.594, 'timezone': 'Asia/Colombo', 'timezone_offset': 19800, 'data': [{'dt': 1602963000, 'sunrise': 1602980650, 'sunset': 1603023660, 'temp': 22.42, 'feels_like': 23.17, 'pressure': 1010, 'humidity': 94, 'dew_point': 21.41, 'clouds': 89, 'wind_speed': 1.44, 'wind_deg': 185, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04n'}]}]}
-----------------------------------
Weather data for 2020-10-18 02:00:00: